In [ ]:
#importing required packages 
import sqlite3
import pandas as pd
import csv
import os
import glob
import string
import csv
from os import rename
import requests
import zipfile

#Creating a sub-directory
staging_dir_name = "staging"
os.mkdir(staging_dir_name)
os.path.isdir(staging_dir_name)

#Downloading zip file from url and unzipping it in the staging directoy
url = "https://data.medicare.gov/views/bg9k-emty/files/0a9879e0-3312-4719-a1db-39fd114890f1?content_type=application%2Fzip%3B%20charset%3Dbinary&filename=Hospital_Revised_Flatfiles.zip"
r = requests.get(url)
zip_file_name = os.path.join(staging_dir_name,"Hospital_Revised_Flatfiles.zip")
zip_file_name
zf = open(zip_file_name,"wb")
zf.write(r.content)
zf.close()
z = zipfile.ZipFile(zip_file_name, "r")
z.extractall(staging_dir_name)
z.close()

#Defining Functions

#The clean function helps in cleaning table names and column names
def clean(x):
    x = x.lower()
    x = x.replace(" ", "_") 
    x = x.replace("-", "_")
    x = x.replace("%", "pct")
    x = x.replace("/", "_")
    if x[0].isalpha() == False:
        x = "t_" + x
    return x

#function to get only csv names from the full path list
def get_csv_names(a):
    #s = str(a)
    x = a.rfind("\\")
    return(x + 1)

#function to remove single quotes from names
def removequotes(x):
    z=x.replace("'",'')
    return (z)

#reading all the csv files
rwd = os.getcwd()
swd = os.getcwd()+"/staging"
os.chdir(swd)
file = glob.glob("*.csv")

#removing corrupt file
if('FY2015_Percent_Change_in_Medicare_Payments.csv' in file):
    os.remove('FY2015_Percent_Change_in_Medicare_Payments.csv')
    files.remove('FY2015_Percent_Change_in_Medicare_Payments.csv')
    
#getting csv names after cleaning it
list_files = glob.glob("*.csv")
for lname in list_files:
    rename(lname, lname.replace(lname,clean(lname)))
list_files = glob.glob("*.csv")


#getting full path along with the csv names
full_path_list = []
for a in list_files:
    full_path_list.append(os.getcwd()+"\\"+a)
    
#going back to local directory
os.chdir("..")

#Changing the encoding of files to make it python compatible
for enc in full_path_list:
    inp_files = open(enc,"rt", encoding='cp1252')
    i_data = inp_files.read()
    inp_files.close()
    out_files = open(enc,"wt", encoding='utf-8')
    for c in i_data:
        if c !='\0':
            out_files.write(c)
    out_files.close()

#creating database and setting up a connection    
conn = sqlite3.connect("medicare_hospital_compare.db")
curs = conn.cursor()

#Creating and loading tables
for file_name in full_path_list:
    list1 = []
    tablename = []
    with open(file_name, encoding='utf-8') as csvfile:
        readCSV = csv.reader(csvfile, delimiter = ',')
        for row in readCSV:
            list1.append(row)
    list1[0] = [clean(a) for a in list1[0]]
    x = get_csv_names(file_name)
    tablename = file_name[x:]
    tablename = tablename[:-4]
    tablename = removequotes(tablename)
    curs.execute("create table if not exists " +tablename + " (" + ",".join(str(a + " text") for a in list1[0])+");")
    #i=1
    #j=len(list1)
    if (tablename != 'psi_april2017') and (tablename != 'mort_readm_april2017'):
        i=1
        j=len(list1)
        while (i<j):
            list1[i]=[removequotes(f) for f in list1[i]]
            curs.execute("insert into "+  tablename +" values("+ ','.join(str("'"+k+"'") for k in list1[i]) +')')
            i+=1;
    else:
        i=1
        j=len(list1)-2
        while (i<(j)):
            list1[i]=[removequotes(f) for f in list1[i]]
            curs.execute("insert into "+ tablename +" values("+ ','.join(str("'"+k+"'") for k in list1[i]) +')')
            i+=1;
    list1[:] = []
conn.commit()
conn.close()

#Part 1 END

#Part 2 START

#gettting the hospital ranking file from url and reading it
url_1 = "http://kevincrook.com/utd/hospital_ranking_focus_states.xlsx"
x1 = pd.ExcelFile(url_1)

#connecting to the database
conn = sqlite3.connect("medicare_hospital_comapre.db")
curs = conn.cursor()

#getting sheets as dataframes
sheet = x1.sheet_names
hospital_ranking = x1.parse(sheet[0])
focus_states = x1.parse(sheet[1])

# Reading the required csv files with rankings and making it ready to use
query = ("SELECT * FROM hospital_general_information;")
result = pd.read_sql_query(query,conn)
result['provider_id'] = pd.to_numeric(result['provider_id'])
result.rename(columns={'provider_id':'Provider ID'}, inplace = True)
df1 = result[["Provider ID","hospital_name", "city", "state", "county_name"]]

#Merging 2 data frames and sorting it
National_Hospital_Rannkin = hospital_ranking.merge(df1, left_on = 'Provider ID', right_on = 'Provider ID', how = 'left', left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=False, indicator=False)
sorted_national_ranking = National_Hospital_Rannkin.reindex(National_Hospital_Rannkin.Ranking.sort_values().index)
sorted_national_ranking.drop('Ranking', axis = 1, inplace = True)
national_top100 = sorted_national_ranking.head(100)

#Creating excel file with national hospital ranking
national_top100.columns = ['Provider ID', 'Hospital Name', 'City','State','County']
writer = pd.ExcelWriter('hospital_ranking.xlsx', engine='xlsxwriter')
national_top100.to_excel(writer, sheet_name = 'Nationwide', index = False)

# Getting the top 100 state wise hospital ranking for states of interest and writing it in excel
focus_states.sort_values(by = 'State Name', ascending = True, inplace = True, axis = 0)
for index, row in focus_states.iterrows():
    df103 = National_Hospital_Rannkin.loc[National_Hospital_Rannkin['state'] == row['State Abbreviation']]
    df103 = df103.head(100)
    # Convert the dataframe to an XlsxWriter Excel object.
    df103.to_excel(writer, sheet_name=row['State Name'], index = False)

writer.close()
writer.save()

#Part 2 END

#Part 3 START
#Quering and reading the required file
score_query = ("select state, measure_id, measure_name, cast(score as int) score from timely_and_effective_care___hospital where score not in ('Not Available');")
score = pd.read_sql_query(score_query,conn)

#grouping and creating aggregate measures
nation_score = score.sort_values('measure_id').groupby(['measure_id','measure_name']).agg({'score':['min','max','mean','std']})
nation_score = pd.DataFrame(nation_score.to_records())


#Creating columns and writiing the data to excel nationwide
nation_score.columns=['Measure ID', 'Measure Name', 'Minimum','Maximum','Average','Standard Deviation']
writer = pd.ExcelWriter('measure_statistics.xlsx', engine = 'xlsxwriter')
nation_score.to_excel(writer,'Nationwide', index = False)

#Getting the top 100 state wise hospital measures for states of interest and writing it in excel
for index, row in focus_states.iterrows():
    df12 = score.loc[score['state'] == row['State Abbreviation']]
    df12 = df12.sort_values('measure_id').groupby(['measure_id','measure_name']).agg({'score':['min','max','mean','std']})
    df12 = pd.DataFrame(df12.to_records())
    df12.columns=['Measure ID', 'Measure Name', 'Minimum','Maximum','Average','Standard Deviation']
    df12.to_excel(writer, sheet_name=row['State Name'], index = False)
    
writer.close()
writer.save()

#Part 3 END
#END